<a href="https://colab.research.google.com/github/gazon1/Recco-challenge/blob/master/LSTM_from_kaggle_baseline_Oleg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

from sklearn.metrics import # Rekko challenge 2019

```
                           /$$$$$$$  /$$$$$$$$ /$$   /$$ /$$   /$$  /$$$$$$ 
                          | $$__  $$| $$_____/| $$  /$$/| $$  /$$/ /$$__  $$
                          | $$  \ $$| $$      | $$ /$$/ | $$ /$$/ | $$  \ $$
                          | $$$$$$$/| $$$$$   | $$$$$/  | $$$$$/  | $$  | $$
                          | $$__  $$| $$__/   | $$  $$  | $$  $$  | $$  | $$
                          | $$  \ $$| $$      | $$\  $$ | $$\  $$ | $$  | $$
                          | $$  | $$| $$$$$$$$| $$ \  $$| $$ \  $$|  $$$$$$/
                          |__/  |__/|________/|__/  \__/|__/  \__/ \______/ 
                                                                            
```

Добро пожаловать на соревнование по машинному обучению от онлайн-кинотеатра [Okko](http://okko.tv) Rekko Challenge 2019.

В этом ноутбуке мы покажем вам пример простого но полного решения, от загрузки данных до формирования ответа. Для работы нам понадобятся библиотеки `pandas`, `numpy`, `scipy`, `implicit`, `pprint`, `tqdm`. Установить их в вашем рабочем окружении можно следующей командой.
```
pip install pandas numpy scipy implicit pprint tqdm
```

In [0]:
import os
import json
import pandas as pd
import numpy as np


import tqdm
import scipy.sparse as sp
from sklearn.metrics import average_precision_score as score
import time
from matplotlib import pyplot as plt

import itertools
from pprint import pprint

DATA_PATH = ''
%matplotlib inline

# For Google Colab only:
import sys
sys.path.append('/content/gdrive/My Drive/okko competition')
#from reco_utils.recommender.sar.sar_singlenode import SARSingleNode
from google.colab import drive
drive.mount('/content/gdrive')

DATA_PATH = 'gdrive/My Drive/okko competition/data'
CODE_PATH = 'gdrive/My Drive/okko competition/code'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#Hyperparametrs:

WATCHED_TIME = 600 # выкидываем передачи, просмотренные меньше WATCHED_TIME. ЧТобы ничего не выкидывать - ставим 0
MIN_RATING = 3 # ниже этого рейтинга все выкидываем


### Загрузка данных

`catalogue.json` содержит анонимизированную метаинформацию о доступных в сервисе фильмах и сериалах.

In [0]:
with open(os.path.join(DATA_PATH, 'catalogue.json'), 'r') as f:
    catalogue = json.load(f)
    
catalogue = {int(k): v for k, v in catalogue.items()}
catalogue = pd.DataFrame.from_dict(catalogue,'index').reset_index()
# av_hot - нужно для валидации. Эта колонка равна 1, если фильм или сериал может быть потребленным
catalogue['av_hot'] = catalogue['availability'].apply(lambda x: 1 if x == [] or 'subscription' in x else 0)
catalogue = catalogue.rename(index=str, columns={'index':'element_uid'})

In [0]:
catalogue.head()

,element_uid,type,availability,duration,feature_1,feature_2,feature_3,feature_4,feature_5,attributes,av_hot
0,0,movie,[],80,2.912198e+07,0.575260,0,1.128332,0.654707,"[31115, 6713, 10906, 31116, 31117, 270, 24431,...",1
1,1,movie,"[purchase, rent]",120,6.610431e+06,0.773224,3,1.112014,0.654707,"[2786, 385, 2799, 3730, 886, 7, 11700, 42, 20,...",0
2,2,movie,[],80,1.315874e+07,0.699502,0,1.110127,0.680410,"[31442, 31443, 31444, 31445, 113, 31446, 42, 3...",1
3,3,series,[],20,4.157712e+07,0.702981,0,1.141929,0.654707,"[34361, 34362, 23033, 14887, 270, 20089, 43, 25]",1
4,4,movie,"[purchase, rent, subscription]",70,3.999579e+07,0.626596,8,1.130076,0.592716,"[26732, 26733, 26734, 9367, 7792, 336, 26735, ...",1


 - `attributes` — мешок атрибутов
 - `availability` — доступность (может содержать значения `purchase`, `rent` и `subscription`)
 - `duration` — длительность в минутах, округлённая до десятков (продолжительность серии для сериалов и многосерийных фильмов)
 - `feature_1..5` — пять анонимизированных вещественных и порядковых признаков
 - `type` — принимает значения `movie`, `multipart_movie` или `series`

`test_users.json` содержит список пользователей, для которых необходимо построить предсказание

In [0]:
with open(os.path.join(DATA_PATH, 'test_users.json'), 'r') as f:
    test_users = set(json.load(f)['users'])
len(test_users)

50000

`transactions.csv` — список всех транзакций за определённый период времени

In [0]:
%%time
transactions = pd.read_csv(
    os.path.join(DATA_PATH, 'transactions.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'consumption_mode': 'category',
        'ts': np.float64,
        'watched_time': np.uint64,
        'device_type': np.uint8,
        'device_manufacturer': np.uint8
    }
)

CPU times: user 7.71 s, sys: 582 ms, total: 8.29 s
Wall time: 10.6 s


In [0]:
%%time
ratings = pd.read_csv(
    os.path.join(DATA_PATH, 'ratings.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'ts': np.float64,
        'rating': np.uint8
    }
)

CPU times: user 251 ms, sys: 14 ms, total: 265 ms
Wall time: 599 ms


In [0]:
%%time
bookmarks = pd.read_csv(
    os.path.join(DATA_PATH, 'bookmarks.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'ts': np.float64
    }
)

CPU times: user 458 ms, sys: 12.6 ms, total: 471 ms
Wall time: 1.6 s


In [0]:
bookmarks['rating'] = 1
_bookmarks = bookmarks[['rating', 'user_uid', 'element_uid','ts']]

_ratings = ratings[ratings['rating']>5]
_ratings['rating'] = 1

_transaction = transactions[['element_uid', 'user_uid','ts','watched_time']]
_transaction = _transaction[(_transaction['watched_time']>700)]
_transaction['rating'] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [0]:
all_ratings = pd.concat([ _transaction[['user_uid', 'element_uid', 'rating', 'ts']],_bookmarks[['user_uid', 'element_uid', 'rating', 'ts']]])
all_ratings = pd.concat([all_ratings, _ratings[['user_uid', 'element_uid', 'rating','ts']]])
all_ratings = all_ratings.drop_duplicates(['user_uid','element_uid'],keep='first')
all_ratings = all_ratings.sort_values(by=['ts'])
all_ratings= all_ratings.rename(index=str,columns={'element_uid':'itemID','user_uid':'userID','ts':'timestamp'})

In [0]:

all_ratings.head(3)

,userID,itemID,rating,timestamp
9643011,283774,6189,1,4.173063e+07
9643010,59148,6099,1,4.173063e+07
9643009,50431,8888,1,4.173063e+07


In [0]:
df = pd.merge(all_ratings, catalogue, how='left', left_on='itemID', right_on='element_uid')
del all_ratings
df['ismovie'] = 0
df.loc[df['type']=='movie','ismovie'] = 1
df['isseries'] = 0
df.loc[df['type']=='series','isseries'] = 1
df['ismulti'] = 0
df.loc[df['type']=='multipart_movie','ismulti'] = 1


def func(row):
  return [row['itemID'],row['av_hot'],row['ismovie'],row['isseries'],row['ismulti']]

In [0]:
%%time
# it will take about 6 minutes
df['gen'] = df.apply(func,axis=1)
#df.shape

KeyboardInterrupt: ignored

### Решение

In [0]:
df.itemID = df.itemID.astype('category')
df['categ_id'] = df.itemID.cat.codes + 1
cat_to_element_uid = dict(zip(
    range(1, len(df.itemID.cat.categories) + 1),
    df.itemID.cat.categories
))



In [0]:
df.to_csv(DATA_PATH+'gen.csv')
df = df[['userID','itemID','categ_id','gen']]
df.to_csv(DATA_PATH+'df.csv')

In [0]:
#del all_ratings

df = pd.read_csv(
    os.path.join(DATA_PATH+ 'df.csv')
)

In [0]:
def func(x):
  return x[1:-1].split(',')
df['gen'] = df['gen'].apply(func)

In [0]:
%%time
import tqdm
tqdm.tqdm.pandas()
sequences = df.groupby('userID').agg({'gen': sum})['gen'].progress_apply(list)
# it will take about 2 mins


In [0]:
#type(sequences)
seqq = [sequences.values[0:,][i] for i in range(sequences.shape[0])]
#del sequences

In [0]:
#sequences2use = sequences[len(sequences) >= 20]
#sequences.values[0:,][1]
print(len(seqq))
print(seqq[1])
seqq = np.array(seqq)

496046
['1653', ' 0', ' 1', ' 0', ' 0', '4120', ' 0', ' 1', ' 0', ' 0', '2245', ' 1', ' 1', ' 0', ' 0', '8237', ' 1', ' 1', ' 0', ' 0', '10084', ' 1', ' 1', ' 0', ' 0', '6976', ' 1', ' 1', ' 0', ' 0', '6131', ' 1', ' 1', ' 0', ' 0', '5445', ' 1', ' 1', ' 0', ' 0', '6915', ' 1', ' 1', ' 0', ' 0', '434', ' 1', ' 1', ' 0', ' 0', '6794', ' 1', ' 1', ' 0', ' 0', '7449', ' 1', ' 1', ' 0', ' 0', '8085', ' 1', ' 1', ' 0', ' 0', '4728', ' 0', ' 1', ' 0', ' 0', '1844', ' 1', ' 1', ' 0', ' 0', '7642', ' 1', ' 1', ' 0', ' 0', '9412', ' 1', ' 1', ' 0', ' 0', '9932', ' 0', ' 1', ' 0', ' 0']


In [0]:
maxlen = 18*5 # Length of sequences in X
X = []
y = []

def slice_sequence(seq, num_slices):
  n = len(seq)
  if n > maxlen:
    X.append(seq[:maxlen-1])
    y.append(seq[maxlen])
 # if n == maxlen:
 #   X.append(seq[:maxlen-5])
 #   y.append(seq[maxlen-5])
 #   if seq[maxlen-5] == 0:
 #     print(seq)

  
        
for seq in tqdm.tqdm(seqq):
    if len(seq) <= 5*5:
        slice_sequence(seq, 2)
    elif len(seq) <= 6*5:
        slice_sequence(seq, 2)
    elif len(seq) <= 8*5:
        slice_sequence(seq, 2)
    elif len(seq) <= 12*5:
        slice_sequence(seq, 2)
    elif len(seq) <= 16:
        slice_sequence(seq, 2)
    elif len(seq) <= 20:
        slice_sequence(seq, 2)
    elif len(seq) <= 26:
        slice_sequence(seq, 2)
    else:
        slice_sequence(seq, 2)
        
X = np.ravel(X).reshape((-1, 18, 5))

100%|██████████| 496046/496046 [00:01<00:00, 269585.27it/s]


In [0]:
maxlen = 18 # Length of sequences in X
X = []
y = []

def slice_sequence(seq, num_slices):
    for i in range(1, num_slices):
        X.append(seq[-(i+maxlen): -i])
        y.append(seq[-i])
        
        
sequences2use = sequences[sequences.apply(len) >= 5]
for seq in tqdm.tqdm(sequences2use):
    if len(seq) <= 5:
        slice_sequence(seq, 2)
    elif len(seq) <= 6:
        slice_sequence(seq, 3)
    elif len(seq) <= 8:
        slice_sequence(seq, 4)
    elif len(seq) <= 12:
        slice_sequence(seq, 6)
    elif len(seq) <= 16:
        slice_sequence(seq, 8)
    elif len(seq) <= 20:
        slice_sequence(seq, 11)
    elif len(seq) <= 26:
        slice_sequence(seq, 16)
    else:
        slice_sequence(seq, 23)

100%|██████████| 496046/496046 [00:26<00:00, 19073.78it/s]


In [0]:
lens = [len(x) for x in X]
max(lens), min(lens), np.mean(lens), np.median(lens)
max_features = df.categ_id.unique().size + 1
#del df

In [0]:
from keras.preprocessing.sequence import pad_sequences

# We should pad our sequences with 0 values, so they all will have the same length
X = pad_sequences(X, maxlen=maxlen,padding='post',truncating='post',value=0)
y = np.array(y)
X.shape, y.shape


((8433040, 18), (8433040,))

In [0]:
from keras.layers import Input, Embedding, SpatialDropout1D, Dropout, BatchNormalization, Dense,LSTM, CuDNNGRU#, CuDNNLSTM, CuDNNGRU
from keras.models import Model
from keras import optimizers

# Let's set random seed
import tensorflow as tf
tf.set_random_seed(42)
np.random.seed(42)

In [0]:
max_features_all = df.categ_id.unique().size + 1
maxlen=18
embed_size = 512

DEBUG = True

def lstm129(num_features, seq_len):
    inp = Input(shape=(seq_len,num_features))
    x = inp
    if DEBUG:
      print('input:', x.shape)
#     x = Embedding(max_features_all + 2, 256)(inp)
#     x = SpatialDropout1D(0.05)(x)
    if DEBUG:
      print('after Spatial dropout:', x.shape)
    x = BatchNormalization(epsilon=1e-6,weights=None)(x)
    x = CuDNNGRU(256,return_sequences=False)(x)
    if DEBUG:
      print('after CuDNNGRU:', x.shape)
#     x = Dropout(0.02)(x)
    outp = Dense(max_features_all, activation="softmax")(x)
    if DEBUG:
      print('after Dense:', outp.shape)
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizers.SGD(lr=0.01, \
                                                                    momentum=0.0, decay=0.0, nesterov=True, clipnorm=1.),
                  metrics=['sparse_categorical_accuracy'])
    return model

In [0]:
embed_size = 128

def old_lstm129():
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features_all, embed_size)(inp)
    x = SpatialDropout1D(0.05)(x)
    x = CuDNNGRU(128,return_sequences=False)(x)
    x = Dropout(0.02)(x)
    outp = Dense(max_features, activation="softmax")(x)
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop',
                  metrics=['sparse_categorical_accuracy'])
    return model

In [0]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()


_X = np.where(np.ravel(X) == 0, np.full(np.ravel(X).shape, -1), np.ravel(X)).reshape(X.shape)
_X = np.ravel(_X)
le = preprocessing.LabelEncoder()
le = preprocessing.LabelEncoder()
all_movies = list(set(y).union(set(_X[0::5])))
le.fit(all_movies)

#теперь в _y будут числа от 0 до кол-во фильмов (max_features)
_y = le.transform(y)

_X[0::5] = le.transform(_X[0::5])
_X = _X.reshape((-1,18,5))

In [0]:
model = old_lstm129()
model.fit(_X[:, :, 0], _y, batch_size=2048, epochs=10, verbose=True, validation_split=0.01, shuffle=True)

In [0]:
# раскаменти, чтобы обучить модель с фичами вместо embeding слоя

# start = 0
# end = _X.shape[0]
# DEBUG = False
# model = lstm129(5, 18)

# model.fit(_X[start:end,:18, :], _y[start:end], epochs=10, batch_size=2048)

In [0]:
from keras.models import model_from_json
import keras
def save_model(model, prefix=''):
    model_json = model.to_json()
    with open(os.path.join(CODE_PATH, prefix + "model.json"), "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(os.path.join(CODE_PATH, prefix + "model.h5"))
    print("Saved model to disk")
    
def load_model():
    json_file = open(os.path.join(CODE_PATH, 'model.json'), 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(os.path.join(CODE_PATH, "model.h5"))
    return loaded_model

In [0]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14146287626785672023
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 4273390814707937227
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 10310275435120479926
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11276946637
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1650850629698700918
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [0]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [0]:
#model = lstm129()
#model.load_weights('model128.json')

sequences_test = seqq.apply(lambda x: x[-maxlen:])
sequences_test = seqq.apply(lambda x: [0 for i in range(maxlen - len(x))] + x)
test_users_in_sequences = sorted(set(sequences_test.index) & set(test_users))
X_test = np.array(sequences_test[test_users_in_sequences].tolist())

AttributeError: ignored

In [0]:
%%time
from itertools import chain
batch_size = 2048
n_batches = int(X_test.shape[0]/batch_size) + 1
preds = []

for batch_ind in tqdm.tqdm(range(n_batches)):
    batch = X_test[batch_ind*batch_size: (batch_ind + 1)*batch_size]
    curr_preds = model.predict(batch)
    curr_preds = np.argsort(-curr_preds)[:, :20]
    curr_preds = [[cat_to_element_uid[x] for x in row] for row in curr_preds]
    preds.append([' '.join(map(lambda x: str(x), row)) for row in curr_preds])
    
preds = list(chain(*preds))

100%|██████████| 25/25 [01:00<00:00,  2.01s/it]

CPU times: user 1min 11s, sys: 3.76 s, total: 1min 14s
Wall time: 1min


In [0]:
top_20_videos = [2714, 747, 2639, 3783, 2245, 6127, 10061, 3916,
                 8771, 3336, 7079, 3567, 6195, 4141, 1016,
                 7185, 1004, 9467, 72, 1521]

sam_sub = dict()
for el in test_users:
    sam_sub[el] = top_20_videos

In [0]:
keys = [int(x) for x in sam_sub.keys()]
df = pd.DataFrame(keys,columns=['userID'])
df['itemID'] = df.apply(lambda x: sam_sub[x[0]],axis=1)

In [0]:
df.index = df.userID
df.itemID[test_users_in_sequences] = preds

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [0]:
ans = {}
for el, el1 in df.values:
    try:
        el1 = el1.split(' ')
        for i in range(len(el1)):
            el1[i] = int(el1[i])
        ans[str(el)]= el1
    except:
        ans[str(el)] = el1
    
with open(os.path.join(DATA_PATH, 'my_best.json'), 'w') as f:
    json.dump(ans, f)